In [11]:
# 重启
#!kill -9 -1 
# 切换到自己drive的数据存储目录
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/Colab_Notebooks/Employee_Resignation
!ls
# install and import package
!pip install tpot
from sklearn import preprocessing
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import cross_val_score
from tpot import TPOTRegressor
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import seaborn as sns

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab_Notebooks/Employee_Resignation
Employee_Resignation.py  predict_lr.csv  Resignation_Rate.ipynb  train.csv
prediction.csv		 README.md	 test.csv


## Load data 加载数据

In [0]:
train_data = pd.read_csv('train.csv', index_col='user_id')
train_y = train_data['Attrition'].replace(['No', 'Yes'], [0, 1])
train_x = train_data.drop(columns=['Attrition', 'EmployeeNumber', 
                  'Over18', 'EmployeeCount'])
test_data = pd.read_csv('test.csv', index_col='user_id')
test_x = test_data.drop(columns=['EmployeeNumber', 'Over18', 
                  'EmployeeCount'])

## Explore Data 数据探索

In [0]:
pd.set_option('display.max_columns', None)  # 显示所有列
print('查看数据信息：列名、非空个数、类型等')
print(train_data.info())
print('-' * 30)
print('查看数据摘要')
print(train_data.describe())
print('-' * 30)
print('查看离散数据分布')
print(train_data.describe(include=['O']))
print('-' * 30)
print('查看前5条数据')
print(train_data.head())
print('-' * 30)
print('查看后5条数据')
print(train_data.tail())

## 数据预处理

In [0]:
# 使用one hot编码进行特征值处理 目前最好的结果是0.71
# 特征转换，字符型转成不同列
dvec = DictVectorizer(sparse=False)
train_x = dvec.fit_transform(train_x.to_dict(orient='record'))
test_x = dvec.transform(test_x.to_dict(orient='record'))


In [0]:
# 将离散值转换为数值类型的数据预处理
def extract_features(df):
  # BusinessTravel
  businesstravel_dict = {'Non-Travel':0, 'Travel_Rarely':1, 'Travel_Frequently':2}
  df['BusinessTravel'] = df['BusinessTravel'].map(lambda x: businesstravel_dict[x])
  # Department
  department_dict = {'Sales':0, 'Research & Development':1, 'Human Resources':2}
  df['Department'] = df['Department'].map(lambda x: department_dict[x])
  # EducationField
  educationfield_dict = {'Life Sciences':0, 'Medical':1, 'Marketing':2, 'Technical Degree':3, 'Human Resources':4, 'Other':5}
  df['EducationField'] = df['EducationField'].map(lambda x: educationfield_dict[x])
  # Gender
  gender_dict = {'Male':0, 'Female': 1}
  df['Gender'] = df['Gender'].map(lambda x: gender_dict[x])
  # JobRole
  jobrole_dict = {'Sales Executive':0, 
                  'Research Scientist':1, 
                  'Laboratory Technician':2, 
                  'Manufacturing Director':3, 
                  'Healthcare Representative':4,
                  'Manager':5, 
                  'Sales Representative':6,
                  'Research Director':7,
                  'Human Resources':8
                  }
  df['JobRole'] = df['JobRole'].map(lambda x: jobrole_dict[x])
  # MaritalStatus
  maritalstatus_dict = {'Single':0, 'Married':1, 'Divorced':2}
  df['MaritalStatus'] = df['MaritalStatus'].map(lambda x: maritalstatus_dict[x])
  # OverTime
  overtime_dict = {'Yes':0, 'No':1}
  df['OverTime'] = df['OverTime'].map(lambda x: overtime_dict[x])
  return df
train_x = extract_features(train_x)
test_x = extract_features(test_x)

In [14]:
print(train_x)


         Age  BusinessTravel  ...  YearsSinceLastPromotion  YearsWithCurrManager
user_id                       ...                                               
1374      58               1  ...                        0                     0
1092      45               1  ...                        0                     3
768       40               1  ...                        7                     5
569       36               0  ...                        0                     9
911       25               2  ...                        1                     0
...      ...             ...  ...                      ...                   ...
763       34               1  ...                        0                     0
835       35               1  ...                        1                     4
1216      43               1  ...                        8                     8
559       38               1  ...                        0                     1
684       40               1

In [0]:
# 采用min-max规范化
min_max_scaler = preprocessing.MinMaxScaler()
train_x = min_max_scaler.fit_transform(train_x)
test_x = min_max_scaler.transform(test_x)

## 模型训练与预测

In [16]:
lr = LogisticRegression(class_weight='balanced')
lr.fit(train_x,train_y)
print('LR准确率(基于训练集)： %.4lf' % lr.score(train_x, train_y))
predict_lr = lr.predict(test_x)

df1 = test_data
df1['Attrition'] =predict_lr
df1 = df1['Attrition']
df1.to_csv('predict_lr.csv')

LR准确率(基于训练集)： 0.2326


In [0]:
tpot = TPOTRegressor(generations=5, population_size=50, verbosity=2, random_state=42)
tpot.fit(train_x, train_y)
print(tpot.score(train_x, train_y))

Generation 1 - Current best internal CV score: -0.095178489494318
Generation 2 - Current best internal CV score: -0.09275246814145795
Generation 3 - Current best internal CV score: -0.09232734221869951
Generation 4 - Current best internal CV score: -0.09223478870531687
Generation 5 - Current best internal CV score: -0.09116720411741516

Best pipeline: ExtraTreesRegressor(XGBRegressor(LinearSVR(MaxAbsScaler(input_matrix), C=0.5, dual=True, epsilon=0.01, loss=squared_epsilon_insensitive, tol=1e-05), learning_rate=0.001, max_depth=8, min_child_weight=20, n_estimators=100, nthread=1, objective=reg:squarederror, subsample=0.2), bootstrap=False, max_features=0.55, min_samples_leaf=7, min_samples_split=15, n_estimators=100)
-0.04575698766842926


In [0]:
tpot.export('tpot_resignation_pipeline.py')